In [1]:
import numpy as np
import pyodbc
import pandas as pd
from itertools import combinations
from apriori_python.apriori_algo import *
from apriori_python.utils import *
import ast

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_11756\1088796699.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# test on data of the paper
df1 = pd.read_excel('data/table1.xlsx')
df2 = pd.read_excel('data/table2.xlsx')

In [3]:
res = df1['Item'].tolist()
itemSetList = []
for value in df1['Item']:
    # Remove brackets from the string
    value = value.strip('[]')

    # Split the string by commas (or any other delimiter)
    list_from_string = value.split(',')

    # Convert each element to integers (if needed)
    list_from_string = [item for item in list_from_string]
    itemSetList.append(list_from_string)


In [4]:
global_sup = 0.4  # sigma_g
local_sup = 0.2   # sigma_l 
temporal_sup = 3  # sigma_t 3 days
conf_thres = 0.5  # sigma_c 
sf = 1.3
gf = 0.7
tab1 = df1.values.tolist()
tab2 = df2.values.tolist()

def getGlobalLarge_1_ItemSet(itemSetList):
    C1ItemSet = getItemSetFromList(itemSetList)
    globalItemSetWithSup = defaultdict(int)
    GL1 = getAboveMinSup(C1ItemSet, itemSetList, global_sup, globalItemSetWithSup)
    return GL1

def tolist(series):
    return series.tolist()

def getLocalCandidate_1_d_itemset(GL1):
    sets = [list(s) for s in GL1]
    sets.sort()
    data = []
    Tx = 0
    ls = 0
    count = 0
    for x in sets:
        for y in tab2:
            if set(x).issubset(set(y)):
                LC1 = (y[0],y[1])
                Time = (y[2],y[3])
                Tx = y[3] - y[2] + 1

                for z in tab1:
                    if set(x).issubset(set(z[1])) and z[2] >= Time[0] and z[2] <= Time[1]:
                        count += 1
                
                # Create a dictionary of values
                values = {'LC1': LC1, 'Time': Time, 'Tx': Tx, 'ls': float(count / len(tab1))}
                        
                data.append(values)
                count = 0
    df = pd.DataFrame(data)
    df = df.groupby('LC1').agg({'Time': tolist, 'Tx': 'sum', 'ls': 'sum'}).reset_index()
    data = df.to_dict('records')

    return data

def getLocalLarge_1_ItemSet(GL1, LC1):
    data = []
    for row in LC1:
        if row['ls'] >= local_sup:
            count = 0
            count2 = 0
            print(type(row['LC1'][0]))
            support = GL1[frozenset(row['LC1'][0])]
            for i in tab1:
                timeStamp = i[2]
                for j in row['Time']:
                    
                    if timeStamp >= j[0] and timeStamp <= j[1]:
                        count += 1
                        if set(row['LC1'][0]).issubset(set(i[1])):
                            count2 += 1
            ef = round(count2 / count / support, 2)

            if ef < sf and ef > gf:
                effect = 0
            elif ef >= sf:
                effect = 1
            else:
                effect = -1

            values = {'LL1': [row['LC1']], 'Time': row['Time'], 'rs': float(count2 / count), 's': support, 'EF': ef, 'Effect': effect}        
            data.append(values)
    return data

def inter(list1, list2):
    result = []
    for tuple1 in list1:
        for tuple2 in list2:
            if tuple1[1] > tuple2[0] and tuple1[0] < tuple2[1]:
                intersection = (max(tuple1[0], tuple2[0]), min(tuple1[1], tuple2[1]))
                result.append(intersection)
    return result

def join_lists(list1, list2):
    # Convert lists to sets to remove duplicates
    set1 = set(list1)
    set2 = set(list2)
    
    # Perform union operation to combine sets
    result_set = set1.union(set2)
    
    # Convert the resulting set back to a list
    result_list = list(result_set)
    
    return result_list

def k_minus_one_subsets(input_list, k):
    if k == 1:
        return [[x] for x in input_list]
    else:
        return [list(subset) for subset in combinations(input_list, k-1)]

def get_local_candidate_k_d_itemset(LLk, k):
    # input: LLk  a list (item properties: ['LCk', 'Time', 'Txd', 'ls', 's'])
    # output: LCk  a list
    data = []
    for i in range(len(LLk)):
        for j in range(i+1, len(LLk)):
            rec1 = LLk[i]
            rec2 = LLk[j]

            x1 = rec1['LL' + str(k-1)]
            x2 = rec2['LL' + str(k-1)]

            x1d = [i[0] for i in x1 if len(i) > 1]
            x2d = [i[0] for i in x2 if len(i) > 1]

            if sorted(x1d) == sorted(x2d):
                continue
            
            common = set(x1d).intersection(set(x2d))
            if len(common) != k-2:
                continue
            
            result = inter(rec1['Time'], rec2['Time'])
            # if there is no intersection, skip
            if len(result) == 0:
                continue
            txd = 0
            for time in result:
                txd += time[1] - time[0] + 1

            xd = sorted(join_lists(x1d, x2d))

            lck = sorted(join_lists(rec1['LL' + str(k-1)], rec2['LL' + str(k-1)]))

            subSet = k_minus_one_subsets(lck, k)

            count_k = 0
            for subset in subSet:
                for item in LLk:
                    if set(subset) == set(item['LL' + str(k-1)]):
                        count_k += 1
                        break
            if count_k != len(subSet):
                continue
            
            if txd < temporal_sup:
                s = -1
                ls = -1
            else:
                count = 0
                count2 = 0
                for item in tab1:
                    for hehe in result:
                        if set(xd).issubset(set(item[1])) and ( item[2] >= hehe[0] and item[2] <= hehe[1]):
                            count2 += 1
                    if set(xd).issubset(set(item[1])):
                        count += 1
                s = float(count / len(tab1))
                ls = float(count2 / len(tab1))
            


            value = {'LC' + str(k): lck, 'Time': result, 'Txd': txd, 'ls': ls, 's': s}
            flag = 0
            for d in data:
                if value['LC' + str(k)] == d['LC' + str(k)]:
                    flag = 1
                    break
            if flag == 0:
                data.append(value)
    return data

def get_large_local_k_d_itemset(LCk, k):
    # input: LCk  a list (item properties: ['LCk', 'Time', 'Txd', 'ls', 's'])
    # output: LLk  a list
    data = []
    for item in LCk:
        if item['Txd'] > temporal_sup:
            if item['ls'] >= local_sup:
                LLk = item['LC'+ str(k)]
                time = item['Time']
                count = 0
                count2 = 0
                support = item['s']
                for i in tab1:
                    timeStamp = i[2]
                    for j in item['Time']:
                        if timeStamp >= j[0] and timeStamp <= j[1]:
                            count += 1
                            tempSet = [i[0] for i in item['LC'+ str(k)]]

                            if set(tempSet).issubset(set(i[1])):
                                count2 += 1
                ef = round(count2 / count / support, 2)

                if ef < sf and ef > gf:
                    effect = 0
                elif ef >= sf:
                    effect = 1
                else:
                    effect = -1
                value = {'LL' + str(k): LLk, 'Time': time, 'rs': float(count2 / count), 's': support, 'EF': ef, 'Effect': effect}
                
                data.append(value)
        else:
            continue
    return data


In [5]:
GL1 = getGlobalLarge_1_ItemSet(itemSetList)
LC1 = getLocalCandidate_1_d_itemset(GL1)

LL1 = getLocalLarge_1_ItemSet(GL1, LC1)
LL = [LL1]
k = 2
temp = LL1
while len(temp) != 0:
    LCk = get_local_candidate_k_d_itemset(temp, k)
    LLk = get_large_local_k_d_itemset(LCk, k)
    temp = LLk
    LL = LL + [LLk]
    k += 1

LL = LL[:-1]

for i in LL:
    for j in i:
        print(j)
    print('\n')

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
{'LL1': [('A', 'dr2')], 'Time': [(0, 8), (21, 30)], 'rs': 0.8, 's': 0.71, 'EF': 1.13, 'Effect': 0}
{'LL1': [('B', 'dr1')], 'Time': [(11, 30)], 'rs': 0.4, 's': 0.57, 'EF': 0.7, 'Effect': -1}
{'LL1': [('B', 'dr3')], 'Time': [(0, 10)], 'rs': 1.0, 's': 0.57, 'EF': 1.75, 'Effect': 1}
{'LL1': [('C', 'dr2')], 'Time': [(0, 15)], 'rs': 0.6666666666666666, 's': 0.71, 'EF': 0.94, 'Effect': 0}
{'LL1': [('C', 'dr5')], 'Time': [(23, 30)], 'rs': 1.0, 's': 0.71, 'EF': 1.41, 'Effect': 1}
{'LL1': [('D', 'dr2')], 'Time': [(9, 22)], 'rs': 0.6666666666666666, 's': 0.43, 'EF': 1.55, 'Effect': 1}


{'LL2': [('A', 'dr2'), ('B', 'dr3')], 'Time': [(0, 8)], 'rs': 1.0, 's': 0.42857142857142855, 'EF': 2.33, 'Effect': 1}
{'LL2': [('B', 'dr1'), ('D', 'dr2')], 'Time': [(11, 22)], 'rs': 0.6666666666666666, 's': 0.42857142857142855, 'EF': 1.56, 'Effect': 1}




In [6]:
Mxd = [[]]
dfp = pd.read_excel('data/price.xlsx')
res1 = df1.values.tolist()
res3 = dfp.values.tolist()
k_prime = 1
for subset in LL:
    for itemset in subset:
        temp_list = []
        for transaction in res1:
            LLk = [i[0] for i in itemset['LL' + str(k_prime)]]
            # print(item['Time'][0])
            t1 = itemset['Time'][0][0]
            t2 = itemset['Time'][0][1]
            # print(LLk, transaction[1])
            if set(LLk).issubset(set(transaction[1])) and t1 <= transaction[2] and t2 >= transaction[2]:
                temp = itemset['LL' + str(k_prime)]
                mx = 0
                for item in temp:
                    for price in res3:
                        if item[0] == price[0]:
                            # print(item[1], price[1], price[2])
                            mx += (2 * price[1] - price[2]) * 2
                            print(mx, item[1], price[1], price[2])
                            break
                temp_list.append(mx)
            else:
                print('not ok')
    k_prime += 1
    Mxd.append(temp_list)


28 dr2 8 2
28 dr2 8 2
not ok
not ok
not ok
not ok
not ok
not ok
not ok
not ok
10 dr1 4 3
10 dr1 4 3
not ok
not ok
10 dr3 4 3
10 dr3 4 3
not ok
not ok
not ok
not ok
not ok
24 dr2 7 2
not ok
24 dr2 7 2
not ok
not ok
not ok
not ok
not ok
not ok
not ok
not ok
not ok
24 dr5 7 2
24 dr5 7 2
not ok
not ok
not ok
18 dr2 6 3
18 dr2 6 3
not ok
not ok
28 dr2 8 2
38 dr3 4 3
28 dr2 8 2
38 dr3 4 3
not ok
not ok
not ok
not ok
not ok
not ok
not ok
not ok
10 dr1 4 3
28 dr2 6 3
10 dr1 4 3
28 dr2 6 3
not ok
not ok
